# Sentence Variational Adversarial Active Learning (S-VAAL)
@author: Tyler Bikaun

The following notebook intends to flesh out an initial proof concept of the coupling of S-VAE (Bowman <i>et al.</i> 2016; https://arxiv.org/abs/1511.06349) and VAAL (Sinha <i>et al.</i> 2019; https://arxiv.org/abs/1904.00370)

<b>Application:</b> Named Entity Task (NER)

### Standard Imports

In [57]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
import torch.nn.utils.rnn as rnn_utils

### Utility Functions

In [ ]:
config = {'': ''}

### Models
<i>Model architectures</i><br>
<b>SVAE</b> - RNN<br>
<b>Discriminator</b> - FC NN<br>
<b>Task Learner</b> - RNN<br>

#### SVAE

In [89]:
class SVAE(nn.Module):
    """ Sentence Variational Autoencoder (Bowman et al. 2016)"""
    
    def __init__(self, vocab_size, embedding_size):
        super(SVAE, self).__init__()
        
        self.tensor = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.Tensor
        
        self.max_sequence_length = 40    # arg
        self.sos_idx = 0
        self.eos_idx = 1
        self.pad_idx = 2
        self.unk_idx = 3
        
        self.z_dim = 8
        
        self.rnn_type = 'gru'
        self.bidirectional = False
        self.num_layers = 1
        self.hidden_size = 128
        
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.word_dropout_rate = 0.1
        self.embedding_dropout = nn.Dropout(p=0.5)
        
        # set rnn type
        if self.rnn_type == 'gru':
            rnn = nn.GRU
        else:
            raise ValueError()
        
        # init encoder-decoder RNNs (models are identical)
        self.encoder_rnn = rnn(embedding_size,
                               self.hidden_size, 
                               num_layers=self.num_layers,
                               bidirectional=self.bidirectional,
                               batch_first=True)
        self.decoder_rnn = rnn(embedding_size,
                               self.hidden_size, 
                               num_layers=self.num_layers,
                               bidirectional=self.bidirectional,
                               batch_first=True)

        self.hidden_factor = (2 if self.bidirectional else 1) * self.num_layers
        
        # Initialisation of FC layers
        # These go from encoder to latent (z) space
        self.hidden2mean = nn.Linear(self.hidden_size * self.hidden_factor, self.z_dim)
        self.hidden2logv = nn.Linear(self.hidden_size * self.hidden_factor, self.z_dim)
        self.z2hidden = nn.Linear(self.z_dim, self.hidden_size * self.hidden_factor)
        self.outputs2vocab = nn.Linear(self.hidden_size * (2 if self.bidirectional else 1), vocab_size)
    
    
    def forward(self, input_sequence, length):
        """ Forward pass through VAE """
        
        batch_size = input_sequence.size(0)
        
        
        return 0, 0, 0, 0
    
    def reparameterise(self):
        """ Implement reparameterisation trick (Kingma and Welling 2014) """
        
        return 0

In [90]:
# Testing functionality
vocab_size = 100
hidden_size = 128
SVAE = SVAE(vocab_size, hidden_size)

In [91]:
# Test forward pass
random_tensor = torch.randn((10,))
print(SVAE.forward(random_tensor, len(random_tensor)))

(0, 0, 0, 0)


In [85]:
# Reviewing components
print(f'Encoder: {SVAE.encoder_rnn}')
print(f'Decoder: {SVAE.decoder_rnn}')
print(f'hidden2mean: {SVAE.hidden2mean}')
print(f'hidden2logv: {SVAE.hidden2logv}')
print(f'z2hidden: {SVAE.z2hidden}')
print(f'outputs2vocab: {SVAE.outputs2vocab}')

Encoder: GRU(128, 128, batch_first=True)
Decoder: GRU(128, 128, batch_first=True)
hidden2mean: Linear(in_features=128, out_features=8, bias=True)
hidden2logv: Linear(in_features=128, out_features=8, bias=True)
z2hidden: Linear(in_features=8, out_features=128, bias=True)
outputs2vocab: Linear(in_features=128, out_features=100, bias=True)


#### Task Learner

In [46]:
class TaskLearner(nn.Module):
    """ Task learner for NER """
    
    def __init__(self):
        super(TaskLearner, self).__init__()

In [49]:
# Testing functionality
TaskLearner()

TaskLearner()

#### Discriminator

In [42]:
class Discriminator(nn.Module):
    """ Adversary architecture for discriminator module """
    
    def __init__(self, z_dim=8):
        super(Discriminator, self).__init__()
        
        self.z_dim = z_dim    # latent space dimension
        
        self.net = nn.Sequential(
                                nn.Linear(z_dim, 128),
                                nn.ReLU(True),
                                nn.Linear(128, 128),
                                nn.ReLU(True),
                                nn.Linear(128,1),
                                nn.Sigmoid()
                                )
        
        self.weight_init()
        
    def weight_init(self):
        """ Weight initialisation
        
        Using Xavier uniform initialisation rather than Kaiming (I think that is more focused for CV? TODO: investigate)
        See: https://pytorch.org/cppdocs/api/function_namespacetorch_1_1nn_1_1init_1ace282f75916a862c9678343dfd4d5ffe.html
        """
        for block in self._modules:
            for m in self._modules[block]:
                if type(m) == nn.Linear:
                    torch.nn.init.xavier_uniform_(m.weight)
                    m.bias.data.fill_(0.01)
    
    def forward(self, z):
        """ Forward pass through discriminator
        
        Arguments
        --------
            z : tensor
                Tensor derived from SVAE latent space
        """
        return self.net(z)

In [56]:
# Testing functionality
dsc = Discriminator()

# Pass random tensor through forward pass of discriminator
rand_tensor = torch.randn((8,))
print(f'Input: {rand_tensor}')
print(f'Output: {dsc.forward(rand_tensor)}')

Input: tensor([ 0.0312,  1.0590,  0.3862, -0.1136, -1.1375,  1.2425, -1.3496, -0.7329])
Output: tensor([0.5398], grad_fn=<SigmoidBackward>)


### Training Routine
<i> Pseudo code</i>
```python
for epoch in max_epochs:
        train(task learner)
            get preds
            calc loss
            zero grads
            backpropigate loss
            update model parameters
        for step in max_steps:
            train(SVAE)
        for step in max_steps:
            train(discriminator)
```

### Inference Routine

### Sampling Routine